In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as m
from scipy.interpolate import interp1d
from random import random
import scipy.optimize
from pathlib import Path

In [88]:
case = 'Tokovinin'
lf_type = ''
cluster = 'IC2714'
in_file_q = "func_flat.txt"
algo = 'q'

In [89]:
def get_fractions(case):
    if (case == 'Mermilliod'):
        return 2 / 32, 0
    elif (case == 'Tokovinin'):
        return 8 / 41, 0
    elif (case == 'Tokovinin_quadruples'):
        return 8 / 45, 4 / 45
    else:
        raise ValueError
        
def get_header(case):
    if (case == 'Mermilliod'):
        return 2 / 32, 0
    elif (case == 'Tokovinin'):
        return 8 / 41, 0
    elif (case == 'Tokovinin_quadruples'):
        return 8 / 45, 4 / 45
    else:
        raise ValueError

def lummass (M): 
    """gets luminosity from Mass (in SunMass) """
    return pow(10, A * np.log10(M)**2 + B * np.log10(M) + C)

def get_files(cluster):
    if (cluster == 'IC2714'):
        return "IC2714_density.txt", "iso_4e8_IC2714.txt"
    elif (cluster == 'NGC1912'):
        return "NGC1912_density.txt", "iso_2e8_NGC1912.txt"
    elif (cluster == 'NGC2099'):
        return "NGC2099_density.txt", "iso_4.5e8_NGC2099.txt"
    elif (cluster == 'NGC6834'):
        return "NGC6834_density.txt", "iso_0.77e8_NGC6834.txt"
    elif (cluster == 'NGC7142'):
        return "NGC7142_density.txt", "iso_1.6e9_NGC7142.txt"
    else:
        raise ValueError
    
def get_parameters(cluster): 
    """gets cluster's info"""
    if (cluster == 'IC2714'):
        return 10.484, 0.340,1.87
    elif (cluster == 'NGC1912'):
        return 10.294, 0.253,2.1
    elif (cluster == 'NGC2099'):
        return 10.74, 0.301,1.9
    elif (cluster == 'NGC6834'):
        return 11.588, 0.706, 0
    elif (cluster == 'NGC7142'):
        return 10.2, 0.25,2.5
    else:
        raise ValueError
        
def get_lf(filename, lf_type):
    if lf_type == '':
        names = ['J, mag','F','F_low', 'F_high']
    elif lf_type == 'low':
        names = ['J, mag','F_','F', 'F_high']
    elif lf_type == 'high':
        names = ['J, mag','F_','F_low', 'F']
    
    return pd.read_csv(CLUSTERS_DIRECTORY / filename, delimiter='\s+', header=None, names=names)

def to_lf (a, b, c, step, alpha):
    """gets number of stars with definite magnitude"""
    func = interp1d(a, b)
    return func(c) * step * alpha

def to_mf (a, b, c, d): 
    """gets mass of a starS with definite magnitude"""
    func = interp1d(a, b)
    return func(c)*d

def neyman(data_q): 
    """gets q randomly from definite distribution"""
    iter = True
    while iter == True:
        q, Fr = random(), random()
        F = interp1d(data_q['q'], data_q['F'])
        if Fr <= F(q) and q != 0:
            iter = False
            return q
        else:
            iter = True
            
def func(x, lf, q_2, q_3=0, q_4=0):
    """equation to determine Mass (x)"""
    F = np.power(10, A * np.log10(q_2) ** 2 + 2 * A * np.log10(q_2) * np.log10(x) + B * np.log10(q_2))
         
    if q_3 != 0:
         F += np.power(10, A * np.log10(q_3) ** 2 + 2 * A * np.log10(q_3) * np.log10(x) + B * np.log10(q_3))
            
    if q_4 != 0:
         F += np.power(10, A * np.log10(q_4) ** 2 + 2 * A * np.log10(q_4) * np.log10(x) + B * np.log10(q_4))
    
    return np.log10(lummass(x)) + np.log10(1 + F) - np.log10(lf)

In [90]:
A = - 0.705
B = 4.655
C = - 0.025

CLUSTERS_DIRECTORY = Path('clusters/')
ISOCHRONES_DIRECTORY = Path('isochrones/')
RESULTS_DIRECTORY = Path('results/')

In [91]:
num_triples_coef, num_quadruples_coef = get_fractions(case)
in_file_lf, in_file_is = get_files(cluster)

In [92]:
data_lf = get_lf(in_file_lf, lf_type)
data_is = pd.read_csv(ISOCHRONES_DIRECTORY / in_file_is, delimiter='\s+')
data_q = pd.read_csv(in_file_q, delimiter=' ', header=None, names = ['q','F'])
iso_young = pd.read_csv(ISOCHRONES_DIRECTORY / "iso_4e7.txt", delimiter='\s+')

In [93]:
dist_mod, EBV, Jlim = get_parameters(cluster)

data_lf['Jabs, mag'] = data_lf['J, mag']- 2.43 * 0.37 * EBV - dist_mod

step = (data_lf['Jabs, mag'].iloc[-1] - data_lf['Jabs, mag'].iloc[0]) / 30
main_table = pd.DataFrame(np.linspace(data_lf['Jabs, mag'].iloc[0], data_lf['Jabs, mag'].iloc[-1], num = 31, dtype=None)+step/2)

main_table.rename(columns={0 : 'Jabs'}, inplace = True)
main_table = main_table.drop(main_table.index[30])

#getting the mass of the cluster without any binary
main_table['Num_all_ss'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, 1))
main_table['mass_all_ss'] = to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'], main_table['Num_all_ss'])
MASS_CLUSTER_OLD = main_table['mass_all_ss'].sum()

In [94]:
#for same masses
if algo == 's':
    results = pd.DataFrame(data = {'ALPHA': [0], 'MASS CLUSTER NEW': [0], 'RATIO': [0], 'RATIO SD': [0]})
    for ALPHA in np.arange (0., 1., 0.1):
        print (ALPHA)
        MASS_BINARIES = 0
        
    #getting the mass of binaries in the cluster
        main_table['Num_tri'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, ALPHA*8/45))
        main_table['Num_qua'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, ALPHA*4/45))
        main_table['Num_bin'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, ALPHA))- main_table['Num_tri'] - main_table['Num_cua']
    
        main_table['Lum0_bin'] = lummass(to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'],1))
        main_table['Lum0_bin'][main_table['Jabs'] < Jlim] = pow(10,
                                                            to_mf(iso_young['Mass'], iso_young['logL'],
                                                                  to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'],1),1))
        
        main_table['mass_all_bs'] = to_mf(data_is['logL'], data_is['Mass'], np.log10(main_table['Lum0_bin']/2), main_table['Num_bin'])*2
        main_table['mass_all_ts'] = to_mf(data_is['logL'], data_is['Mass'], np.log10(main_table['Lum0_bin']/3), main_table['Num_tri'])*3
        main_table['mass_all_qs'] = to_mf(data_is['logL'], data_is['Mass'], np.log10(main_table['Lum0_bin']/4), main_table['Num_qua'])*4
    
        MASS_BINARIES = main_table['mass_all_bs'].sum() +main_table['mass_all_ts'].sum() + main_table['mass_all_qs'].sum()
        
    #getting the mass of single stars in the cluster
        main_table['Num_part_ss'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, 1)) - np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, ALPHA))
        main_table['mass_part_ss'] = to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'], main_table['Num_part_ss'])
        MASS_SINGLES = main_table['mass_part_ss'].sum()
   
        MASS_CLUSTER_NEW = MASS_BINARIES + MASS_SINGLES;
        ratio = MASS_CLUSTER_NEW / MASS_CLUSTER_OLD
        results = results.append(pd.Series([ALPHA, MASS_CLUSTER_NEW, np.mean(ratio), np.std(ratio)], index =['ALPHA', 'MASS CLUSTER NEW', 'RATIO', 'RATIO SD']), ignore_index=True)

    results = results.drop(0)
    results = results.round({'ALPHA' : 1, 'MASS CLUSTER NEW' : 2, 'RATIO' : 4, 'RATIO SD' : 4})
    results.to_csv(f"alpha_{in_file_lf.split('_')[0]}_same_4.txt"  , sep='\t', index=False, mode='w')

In [95]:
# for masses by q
if algo == 'q':
    results = pd.DataFrame(data = {'ALPHA': [0], 'MASS CLUSTER NEW': [0], 'RATIO': [0], 'RATIO SD': [0]})
    for ALPHA in np.arange (0., 1., 0.1):
        print (ALPHA)
        ratio = []
        for i in range(0, 30):
            MASS_MULTIPLES = 0
            
        
            #getting the mass of binaries in the cluster
            
            main_table['Num_quad'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], 
                                                   step, ALPHA) * num_quadruples_coef)
            
            main_table['Num_tr'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], 
                                                   step, ALPHA) * num_triples_coef)
            
            main_table['Num_bin'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], 
                                                   step, ALPHA)) - main_table['Num_tr'] - main_table['Num_quad']
            
            main_table['Lum0_bin'] = lummass(to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'],1))
        
            main_table['Lum0_bin'][main_table['Jabs'] < Jlim] = pow(10, to_mf(iso_young['Mass'], iso_young['logL'],to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'],1),1))
        
            data_binaries = main_table.loc[main_table.index.repeat(main_table['Num_bin'].astype(int))]
            data_binaries.index = pd.RangeIndex(len(data_binaries.index))
        
            data_triples = main_table.loc[main_table.index.repeat(main_table['Num_tr'].astype(int))]
            data_triples.index = pd.RangeIndex(len(data_triples.index))
            
            data_quadruples = main_table.loc[main_table.index.repeat(main_table['Num_quad'].astype(int))]
            data_quadruples.index = pd.RangeIndex(len(data_quadruples.index))
            
            for star in range (0, len(data_quadruples)):
                q_2 = neyman(data_q)
                q_3 = neyman(data_q)
                q_4 = neyman(data_q)
                MASS_MULTIPLES += scipy.optimize.brentq(func, 0.001, 10, args = (data_quadruples['Lum0_bin'].iloc[star], q_2, q_3, q_4)) \
                                                                                * (1 + q_2 + q_3 + q_4)
            
            for star in range (0, len(data_triples)):
                q_2 = neyman(data_q)
                q_3 = neyman(data_q)
                MASS_MULTIPLES += scipy.optimize.brentq(func, 0.001, 10, args = (data_triples['Lum0_bin'].iloc[star], q_2, q_3)) \
                                                                                * (1 + q_2 + q_3)

            for star in range (0, len(data_binaries)):
                q = neyman(data_q)
                MASS_MULTIPLES += scipy.optimize.brentq(func, 0.001, 10, args = (data_binaries['Lum0_bin'].iloc[star],q)) * (1 + q)
            
            #getting the mass of single stars in the cluster
            main_table['Num_part_ss'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, 1)) \
                                        - np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, ALPHA))
            main_table['mass_part_ss'] = to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'], main_table['Num_part_ss'])
            MASS_SINGLES = main_table['mass_part_ss'].sum()
   
            MASS_CLUSTER_NEW = MASS_MULTIPLES + MASS_SINGLES
            ratio.append(MASS_CLUSTER_NEW / MASS_CLUSTER_OLD)
        
        results = results.append(pd.Series([ALPHA, MASS_CLUSTER_NEW, np.mean(ratio), np.std(ratio)], index=['ALPHA', 'MASS CLUSTER NEW', 'RATIO', 'RATIO SD']), ignore_index=True)

    results = results.drop(0)
    results = results.round({'ALPHA' : 1, 'MASS CLUSTER NEW' : 2, 'RATIO' : 4, 'RATIO SD' : 4})
    results.to_csv(RESULTS_DIRECTORY / f"alpha_{cluster}_{in_file_q.split('.txt')[0]}_{case}.txt"  , sep='\t', index=False)

0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9
